In [10]:
import pandas as pd
from readii.process.subset import selectByColumnValue
from readii.process.label import setPatientIdAsIndex
from readii.io.loaders.general import loadFileToDataFrame
from pathlib import Path
import numpy as np

In [2]:
clinical_file = Path("../data/rawdata/TCIA_HEAD-NECK-RADIOMICS-HN1/clinical/Copy of HEAD-NECK-RADIOMICS-HN1 Clinical data updated July 2020 2.csv")

In [3]:
raw_clinical_data = loadFileToDataFrame(clinical_file)

In [4]:
raw_clinical_data

,id,index_tumour_location,age_at_diagnosis,biological_sex,performance_status_ecog,overall_hpv_p16_status,clin_t,clin_n,clin_m,ajcc_stage,...,event_overall_survival,overall_survival_in_days,event_recurrence_metastatic_free_survival,recurrence_metastatic_free_survival_in_days,event_local_recurrence,local_recurrence_in_days,event_locoregional_recurrence,locoregional_recurrence_in_days,event_distant_metastases,distant_metastases_in_days
0,HN1004,oropharynx,56,male,1.0,negative,4,2,0,iva,...,1,3193,0,3193,0,3193,0,3193,0,3193
1,HN1006,oropharynx,63,female,1.0,negative,1,2,0,iva,...,0,2805,0,1940,0,1940,0,1940,0,1940
2,HN1022,oropharynx,56,male,0.0,positive,1,0,0,i,...,0,3198,0,1269,0,1269,0,1269,0,1269
3,HN1026,larynx,67,male,0.0,NaN,4,0,0,iva,...,0,3244,1,600,1,600,1,600,0,1315
4,HN1029,oropharynx,54,female,0.0,positive,1,2,0,iva,...,0,2925,0,2101,0,2101,0,2101,0,2101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,HN1950,oropharynx,66,male,1.0,NaN,2,1,0,iii,...,1,330,1,278,1,278,1,278,0,280
133,HN1954,oropharynx,77,female,1.0,negative,2,0,0,ii,...,1,4373,0,2541,0,2541,0,2541,0,2541
134,HN1968,larynx,47,male,0.0,NaN,4,0,0,iva,...,0,3266,1,1196,0,1337,1,1196,0,1337
135,HN1987,oropharynx,66,male,0.0,negative,4,3,0,ivb,...,1,716,0,716,0,716,0,716,0,716


In [11]:
include={'index_tumour_location': ["oropharynx"]}
exclude={'overall_hpv_p16_status': [np.nan]}

indexed_clinical_data = setPatientIdAsIndex(raw_clinical_data, "id")
opc_clinical_data = selectByColumnValue(indexed_clinical_data, include_col_values=include, exclude_col_values=exclude)

In [12]:
opc_clinical_data

,index_tumour_location,age_at_diagnosis,biological_sex,performance_status_ecog,overall_hpv_p16_status,clin_t,clin_n,clin_m,ajcc_stage,pretreat_hb_in_mmolperlitre,...,event_overall_survival,overall_survival_in_days,event_recurrence_metastatic_free_survival,recurrence_metastatic_free_survival_in_days,event_local_recurrence,local_recurrence_in_days,event_locoregional_recurrence,locoregional_recurrence_in_days,event_distant_metastases,distant_metastases_in_days
id,,,,,,,,,,,,,,,,,,,,,
HN1004,oropharynx,56,male,1.0,negative,4,2,0,iva,7.1,...,1,3193,0,3193,0,3193,0,3193,0,3193
HN1006,oropharynx,63,female,1.0,negative,1,2,0,iva,NaN,...,0,2805,0,1940,0,1940,0,1940,0,1940
HN1022,oropharynx,56,male,0.0,positive,1,0,0,i,NaN,...,0,3198,0,1269,0,1269,0,1269,0,1269
HN1029,oropharynx,54,female,0.0,positive,1,2,0,iva,7.4,...,0,2925,0,2101,0,2101,0,2101,0,2101
HN1046,oropharynx,66,male,0.0,negative,2,2,0,iva,NaN,...,1,828,1,707,0,749,0,749,1,707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HN1910,oropharynx,76,male,0.0,negative,2,0,0,ii,8.9,...,1,4055,1,296,1,296,1,296,0,2784
HN1913,oropharynx,67,male,NaN,negative,2,1,0,iii,8.8,...,1,609,1,448,0,567,1,448,0,567
HN1933,oropharynx,62,female,1.0,negative,1,2,0,iva,8.2,...,1,481,0,412,0,412,0,412,0,412


In [14]:
opc_clinical_data.to_csv("../data/procdata/TCIA_HEAD-NECK-RADIOMICS-HN1/clinical/OPC_only_HEAD-NECK-RADIOMICS-HN1_Clinical_20250818.csv", index=True, index_label='id')